In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import pickle

with open('lob_data.pkl', 'rb') as f:
    lob = pickle.load(f)

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib.colors import LinearSegmentedColormap, TwoSlopeNorm
import ipywidgets as widgets
from IPython.display import display

colors = ['green', 'white', 'red']
custom_cmap = LinearSegmentedColormap.from_list('custom_red_white_green', colors)

group_slider = widgets.IntSlider(min=0, max=11, step=1, description='Battery Size')

battery_size_slider = widgets.SelectionSlider(
    options=[10, 15, 20],  # these are the percentage values you want to select
    description='Solar + Wind Green %',
    value=10  # Initial value must be from the options
)

def update_heatmaps(group_idx, per):
    (optimized_cumulative_battery_mw, optimized_cumulative_wind_gen_mw,
     optimized_cumulative_solar_gen_mw, optimized_cumulative_grid_mw,
     optimized_spill_value, optimized_spill_value_balancing_price) = lob[group_idx]
    
    # Calculate the battery size based on the group index
    battery_size = 30 + 15 * group_idx
    
    # Calculate the vcenter based on the selected percentage
    percent = 263560 * per * 0.01  # Adjusting percentage for vcenter calculation
    norm = TwoSlopeNorm(vmin=0, vcenter=percent, vmax=263560)  # Recalculate the normalization

    # Plot the heatmap for optimized_cumulative_grid_mw
    fig, ax = plt.subplots(figsize=(10, 8))
    heatmap = sns.heatmap(optimized_cumulative_grid_mw, cmap=custom_cmap, norm=norm, annot=False, fmt=".1f", ax=ax)
    
    # Add labels and title with Battery Size and the selected 'per' value
    plt.title(f'Battery Size: {battery_size} MWh - Wind + Solar vs Grid Draw | "Percent" Value: {per}%')
    plt.xlabel('Wind Capacity MWh')
    plt.ylabel('Solar Capacity MW')
    plt.xticks(ticks=np.arange(0, 16), labels=np.arange(0, 301, 20))
    plt.yticks(ticks=np.arange(0, 16), labels=np.arange(0, 301, 20))
    
    # Display placeholders for solar capacity, wind capacity, and spill value
    solar_label = plt.text(-3, 18, "Wind Capacity: ", fontsize=12, color="black", ha="left")
    wind_label = plt.text(-3, 19, "Solar Capacity: ", fontsize=12, color="black", ha="left")
    spill_label = plt.text(-3, 20, "Spill Value (M€): ", fontsize=12, color="black", ha="left")

    def on_click(event):
        if event.inaxes == ax:
            # Convert click coordinates to indices
            x, y = int(event.xdata), int(event.ydata)
            
            solar_capacity = x * 20
            wind_capacity = y * 20
            spill_value = optimized_spill_value_balancing_price[y, x] / 1e6  # Convert to million euro

            solar_label.set_text(f"Solar Capacity: {solar_capacity} MW")
            wind_label.set_text(f"Wind Capacity: {wind_capacity} MW")
            spill_label.set_text(f"Spill Value (M€): {spill_value:.2f}")
            
            fig.canvas.draw()

    fig.canvas.mpl_connect('button_press_event', on_click)

    plt.tight_layout()
    plt.show()

widgets.interact(update_heatmaps, group_idx=group_slider, per=battery_size_slider)


interactive(children=(IntSlider(value=0, description='Battery Size', max=11), SelectionSlider(description='Sol…

<function __main__.update_heatmaps(group_idx, per)>